# PY

In [1]:

"""
# -- --------------------------------------------------------------------------------------------------- -- #
# -- project: A SHORT DESCRIPTION OF THE PROJECT                                                         -- #
# -- script: data.py : python script for data collection                                                 -- #
# -- author: YOUR GITHUB USER NAME                                                                       -- #
# -- license: GPL-3.0 License                                                                            -- #
# -- repository: YOUR REPOSITORY URL                                                                     -- #
# -- --------------------------------------------------------------------------------------------------- -- #
"""

# Load packages
import pandas as pd
import numpy as np
from os import listdir, path
from os.path import isfile, join

# Get the absolute path of the folder
abspath = path.abspath("files/")

# Read all the files within the files folder
files = [f[8:-4] for f in listdir(abspath) if isfile(join(abspath, f))]

# Chronologically ordered files list
files = ["NAFTRAC_" + i.strftime("%Y%m%d") for i in sorted(pd.to_datetime(files))]

# Read and store all the files in a dictionary
data_files = {}
for i in files:
    # Read the file
    data = pd.read_csv("files/" + i + ".csv", skiprows=2, header=0)
    # Select only not null columns
    data = data.loc[:, pd.notnull(data.columns)]
    # Clean the ticker name for later use with yfinance
    data["Ticker"] = [i.replace("*","") for i in data["Ticker"]]
    # Weight as decimal
    data["Peso (%)"] = [i/100 for i in data["Peso (%)"]]
    # Save current file in dictionary of data
    data_files[i] = data



In [2]:
"""
# -- --------------------------------------------------------------------------------------------------- -- #
# -- project: A SHORT DESCRIPTION OF THE PROJECT                                                         -- #
# -- script: functions.py : python script with general functions                                         -- #
# -- author: YOUR GITHUB USER NAME                                                                       -- #
# -- license: GPL-3.0 License                                                                            -- #
# -- repository: YOUR REPOSITORY URL                                                                     -- #
# -- --------------------------------------------------------------------------------------------------- -- #
"""

import pandas as pd
import numpy as np
import yfinance as yf
import time
import datetime

# Get Dates
def f_dates(p_files):
    # Labels for dataframe and yfinance
    t_dates = [i.strftime("%d-%m-%Y") for i in sorted([pd.to_datetime(i[8:]).date() for i in p_files])]

    # For other calculations
    i_dates = [i.strftime("%Y-%m-%d") for i in sorted([pd.to_datetime(i[8:]).date() for i in p_files])]

    # Final data to return
    r_f_dates = {"i_dates": i_dates, "t_dates": t_dates}

    return r_f_dates

#%%
# Get Tickers
def f_tickers(p_archivos, p_data_archivos):
    tickers = []
    for i in p_archivos:
        l_tickers = list(p_data_archivos[i]["Ticker"])
        [tickers.append(i + ".MX") for i in l_tickers]
    global_tickers = np.unique(tickers).tolist()

    # Name adjustment
    global_tickers = [i.replace("GFREGIOO.MX", "RA.MX") for i in global_tickers]
    global_tickers = [i.replace("MEXCHEM.MX", "ORBIA.MX") for i in global_tickers]
    global_tickers = [i.replace("LIVEPOLC.1.MX", "LIVEPOLC-1.MX") for i in global_tickers]

    # Remove problematic tickers and cash entries
    [global_tickers.remove(i) for i in ["MXN.MX", "USD.MX", "KOFL.MX", "KOFUBL.MX",
                                        "BSMXB.MX", "SITESB.1.MX", "NEMAKA.MX", "NMKA.MX"]]

    return global_tickers

#%%
# Get Prices
def f_get_prices(p_tickers, p_fechas):
    # Initial date, no changes
    f_ini = p_fechas[0]

    # Initial date plus 3 days
    f_fin = str(datetime.datetime.strptime(p_fechas[-1], "%Y-%m-%d") + datetime.timedelta(days=3))[:10]

    # Time counter
    inicio = time.time()

    # Yahoo finance data download
    data = yf.download(p_tickers, start=f_ini, end=f_fin, actions=False, group_by="close",
                       interval="1d", auto_adjust=False, prepost=False, threads=True)

    # Time length of process
    tiempo = "It took", round(time.time() - inicio, 2), "seconds."

    # Morph date column
    data_close = pd.DataFrame({i: data[i]["Close"] for i in p_tickers})

    # We assume NAFRTAC rebalance and Yahoo finance close price times align.

    # Only relevant dates
    ic_fechas = sorted(list(set(data_close.index.astype(str).tolist()) & set(p_fechas)))

    # All prices
    precios = data_close.iloc[[int(np.where(data_close.index == i)[0]) for i in ic_fechas]]

    # Order columns
    precios = precios.reindex(sorted(precios.columns), axis=1)

    return {"precios": precios, "tiempo": tiempo}


In [3]:
"""
# -- --------------------------------------------------------------------------------------------------- -- #
# -- project: A SHORT DESCRIPTION OF THE PROJECT                                                         -- #
# -- script: main.py : python script with the main functionality                                         -- #
# -- author: YOUR GITHUB USER NAME                                                                       -- #
# -- license: GPL-3.0 License                                                                            -- #
# -- repository: YOUR REPOSITORY URL                                                                     -- #
# -- --------------------------------------------------------------------------------------------------- -- #
"""
# %%
# Import other scripts
import data as dt
import functions as fn
import visualizations as vz

# %%
# ---- Base

# Step 1 - Read all the files
data_files = dt.data_files

# Step 2 - Get all the dates
dates = fn.f_dates(p_files=dt.files)

# Display the first 5 dates in the 2 formats
print(dates["i_dates"][0:4])
print(dates["t_dates"][0:4])
#%%
# Step 3 - Get the tickers for the calculations
global_tickers = fn.f_tickers(p_archivos=dt.files, p_data_archivos=data_files)

# Display global tickers
print(global_tickers[0:4])
#%%
# ---- Historical Prices
global_prices = fn.f_get_prices(p_tickers=global_tickers, p_fechas=dates["i_dates"])
precios = global_prices["precios"]

# Show first and last five prices
print(precios.head(5))
print(precios.tail(5))


['2018-01-31', '2018-02-28', '2018-03-28', '2018-04-30']
['31-01-2018', '28-02-2018', '28-03-2018', '30-04-2018']
['AC.MX', 'ALFAA.MX', 'ALPEKA.MX', 'ALSEA.MX']
[*********************100%***********************]  39 of 39 completed

1 Failed download:
-  .MX: No data found, symbol may be delisted
                 AC.MX   ALFAA.MX  ALPEKA.MX   ALSEA.MX    AMXL.MX  \
Date                                                                 
2018-01-31  135.050003  23.360001  26.860001  60.880001  17.440001   
2018-02-28  130.839996  22.389999  24.420000  64.260002  17.340000   
2018-03-28  125.709999  23.290001  25.260000  63.529999  17.250000   
2018-04-30  129.259995  24.010000  26.809999  69.559998  17.309999   
2018-05-31  119.680000  20.480000  29.750000  65.410004  15.500000   

              ASURB.MX  BBAJIOO.MX  BIMBOA.MX  BOLSAA.MX  CEMEXCPO.MX  ...  \
Date                                                                   ...   
2018-01-31  362.220001   39.330002  45.400002  35.61999

# PASIVA

## PRE

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as web

In [5]:
kay = pd.read_csv("files/a/NAFTRAC_20180131.csv", skiprows=2, header=0)

cash = (kay.iloc[35]["Peso (%)"]+kay.iloc[34]["Peso (%)"]+kay.iloc[16]["Peso (%)"]+kay.iloc[10]["Peso (%)"])/100*1000000


kay["Ticker"] = [i.replace("*","") for i in data["Ticker"]]
kay["Ticker"] = [i.replace("LIVEPOLC.1", "LIVEPOLC-1") for i in data["Ticker"]]
    # Weight as decimal
kay["Ticker"] = kay["Ticker"] + '.MX'
kay = kay.drop([10,16,34,35,36])
kay = kay.sort_values(by=['Ticker'])
cash = cash + sum(kay["Peso (%)"]/100*1000000*0.00125)

In [6]:
ic = (kay["Peso (%)"]/100)/kay["Precio"]*1000000
ic.sum(axis = 0)

22929.660681058755

In [7]:
symbols = np.array(kay["Ticker"])
dates = fn.f_dates(p_files=dt.files)
fechi = dates["i_dates"][0:25]
startp = datetime.datetime.strptime(fechi[0], "%Y-%m-%d") - datetime.timedelta(days=1)
weight = np.array(kay["Peso (%)"] )

In [8]:
price_data = web.get_data_yahoo(symbols,
                               start = startp,
                               end = fechi[-1], interval='d')
price_data = price_data["Adj Close"]
price_data.index.strftime("%Y-%m-%d")
price_data = price_data.loc[fechi]

c:\users\aleja\pycharmprojects\myst_amm_lab1\venv\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: 'SITESB.1.MX', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


In [9]:
ret_data = price_data.pct_change()

In [10]:
weighted_returns = (weight * ret_data)

In [11]:
port_ret = weighted_returns.sum(axis=1)/100
capital = 1000000*(1+port_ret)-cash
cumulative_ret = (port_ret+1).cumprod()-1

In [12]:
df_pasiva_a = pd.DataFrame()
df_pasiva_a["Capital"] = capital
df_pasiva_a["Rendimiento"] = port_ret
df_pasiva_a["Rendimiendo Acumulado"] = cumulative_ret
df_pasiva_a

,Capital,Rendimiento,Rendimiendo Acumulado
Date,,,
2018-01-31,9.539060e+05,0.000000,0.000000
2018-02-28,9.155323e+05,-0.038374,-0.038374
2018-03-28,9.334009e+05,-0.020505,-0.058092
2018-04-30,9.828142e+05,0.028908,-0.030863
2018-05-31,8.824988e+05,-0.071407,-0.100066
2018-06-29,1.020313e+06,0.066407,-0.040305
2018-07-31,9.971122e+05,0.043206,0.001160
2018-08-31,9.579067e+05,0.004001,0.005165
2018-09-28,9.481644e+05,-0.005742,-0.000606


## POST